In [1]:
# импорт библиотек
import numpy as np # для матричных вычислений
import pandas as pd # для анализа и предобработки данных

from sklearn.model_selection import train_test_split # сплитование выборки

from sklearn import linear_model # линейные модели
from sklearn import ensemble # ансамбли

from sklearn import metrics #метрики
from sklearn.model_selection import cross_val_score # для кросс-валидации

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
import optuna

In [2]:
# загрузка данных
data = pd.read_csv('_train_sem09 (1).csv')
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Создаем матрицу наблюдений  𝑋  и вектор ответов  𝑦

In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

Разделяем выборку на тренировочную и тестовую в соотношении 80/20. Для сохранения соотношений целевого признака используем параметр stratify (стратифицированное разбиение).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

# <center> Логистическая регрессия

Обучим модель с гиперпараметрами по умолчанию и рассчитаем метрику f1

In [5]:
# создаем объект класса Логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
# обучаем модель
log_reg.fit(X_train, y_train)
y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


##  GridSearchCV

In [6]:
param_grid = [{'penalty': ['l1', 'l2'] ,# тип регурялизации
              'solver': ['liblinear', 'saga'], # алгоритм оптимизации
              'C': [0.01, 0.05, 0.08, 0.1, 0.15, 0.3, 0.5, 0.7, 0.9, 1] # уровень силы регурялизации
              }             
             ]
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, # генератор случайных чисел
        max_iter=1000 # количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

Wall time: 7min 58s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.08, 'penalty': 'l2', 'solver': 'saga'}


**Удалось немного улучшить метрику**

##  RandomizedSearchCV

In [7]:
param_grid = {'penalty': ['l1', 'l2'] ,# тип регурялизации
              'solver': ['liblinear', 'saga'], # алгоритм оптимизации
               'C': list(np.linspace(0.01, 1, 10, dtype=float))}, # уровень силы регурялизации
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=1000), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 30, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

Wall time: 6min 32s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'solver': 'saga', 'penalty': 'l2', 'C': 0.12}


**Удалось улучшить метрику немного быстрее, чем методом GridSearchCV**

## Optuna

In [10]:
random_state = 42

# функция минимизации
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров
  penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])
  solver = trial.suggest_categorical('solver', ['liblinear', 'saga'])
  C = trial.suggest_float('C', 0.01, 1,)

  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  # применяем кросс-валидацию
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [11]:
%%time
# cоздаем объект исследования
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров
study.optimize(optuna_lr, n_trials=20)

[I 2022-07-14 00:50:02,162] A new study created in memory with name: LogisticRegression
[I 2022-07-14 00:50:04,293] Trial 0 finished with value: 0.7817114644693265 and parameters: {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.12479089948812304}. Best is trial 0 with value: 0.7817114644693265.
C:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-07-14 00:50:24,248] Trial 1 finished with value: 0.7795877974014873 and parameters: {'penalty': 'l2', 'solver': 'saga', 'C': 0.43556631890969055}. Best is trial 0 with value: 0.7817114644693265.
C:\anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-07-14 00:50:51,509] Trial 2 finished with value: 0.7776086287240973 and parameters: {'penalty': 'l1', 'solver': 'saga', 'C': 0.42480208596441077}. Best is trial 

Wall time: 1min 56s


In [12]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'liblinear', 'C': 0.026985557254051287}
f1_score на тестовом наборе: 0.79


**Удалось улучшить метрику за меньшее время по сравнению с GridSearchCV и RandomizedSearchCV**

# <center> Случайный лес

Обучим модель с гиперпараметрами по умолчанию и рассчитаем метрику f1

In [13]:
# создаем объект класса Случайный лес
rf = ensemble.RandomForestClassifier(random_state=42)

# обучаем модель
rf.fit(X_train, y_train)
# выводим значение метрики 
y_test_pred = rf.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.81


## RandomizedSearchCV

In [14]:
param_grid = {'n_estimators': list(range(100, 200, 30)),
              'min_samples_leaf': [5, 7],
              'max_depth': list(range(10, 25, 5))
              }
            
random_search_forest = RandomizedSearchCV(
    estimator=ensemble.RandomForestClassifier(random_state=42), 
    param_distributions=param_grid, 
    cv=5,
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search_forest.fit(X_train, y_train) 
y_train_pred = random_search_forest.predict(X_train)
y_test_pred = random_search_forest.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_forest.best_params_))

Wall time: 20.9 s
f1_score на тестовом наборе: 0.82
Наилучшие значения гиперпараметров: {'n_estimators': 130, 'min_samples_leaf': 5, 'max_depth': 20}


**Удалось улучшить метрику**

## Hyperopt

In [15]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

In [16]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
              'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    
    # применяем кросс-валидацию
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [18]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 20/20 [02:22<00:00,  7.13s/trial, best loss: -0.8080577050312506]
Наилучшие значения гиперпараметров {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
Wall time: 2min 22s


In [19]:
# рассчитаем f1 для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


In [20]:
# продолжаем подбор
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=40, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|███████████████████████████████████████████████| 40/40 [02:58<00:00,  8.93s/trial, best loss: -0.8120423125971236]
Наилучшие значения гиперпараметров {'max_depth': 20.0, 'min_samples_leaf': 2.0, 'n_estimators': 160.0}


In [21]:
# пересчитаем f1 для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.83


**Удалось ещё улучшить результат**

## Optuna

In [24]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
    
  # применяем кросс-валидацию
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

In [25]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-07-14 01:20:16,465] A new study created in memory with name: RandomForestClassifier
[I 2022-07-14 01:20:22,420] Trial 0 finished with value: 0.795046795903416 and parameters: {'n_estimators': 117, 'max_depth': 17, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.795046795903416.
[I 2022-07-14 01:20:30,285] Trial 1 finished with value: 0.8069717805146268 and parameters: {'n_estimators': 132, 'max_depth': 26, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.8069717805146268.
[I 2022-07-14 01:20:37,634] Trial 2 finished with value: 0.8128575206918857 and parameters: {'n_estimators': 103, 'max_depth': 12, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.8128575206918857.
[I 2022-07-14 01:20:46,529] Trial 3 finished with value: 0.7927257982144186 and parameters: {'n_estimators': 175, 'max_depth': 18, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8128575206918857.
[I 2022-07-14 01:20:53,961] Trial 4 finished with value: 0.7988019937547387 and parameters: {'n_

Wall time: 2min 32s


In [26]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 187, 'max_depth': 16, 'min_samples_leaf': 3}
f1_score на тестовом наборе: 0.82


In [27]:
%%time
#продолжим подбор 
study.optimize(optuna_rf, n_trials=20)

[I 2022-07-14 01:24:09,349] Trial 20 finished with value: 0.8110540184428856 and parameters: {'n_estimators': 199, 'max_depth': 17, 'min_samples_leaf': 3}. Best is trial 19 with value: 0.8148591363142641.
[I 2022-07-14 01:24:21,616] Trial 21 finished with value: 0.8108658778192256 and parameters: {'n_estimators': 185, 'max_depth': 14, 'min_samples_leaf': 2}. Best is trial 19 with value: 0.8148591363142641.
[I 2022-07-14 01:24:31,268] Trial 22 finished with value: 0.8057383176304042 and parameters: {'n_estimators': 159, 'max_depth': 12, 'min_samples_leaf': 3}. Best is trial 19 with value: 0.8148591363142641.
[I 2022-07-14 01:24:42,645] Trial 23 finished with value: 0.8080664171298781 and parameters: {'n_estimators': 186, 'max_depth': 16, 'min_samples_leaf': 5}. Best is trial 19 with value: 0.8148591363142641.
[I 2022-07-14 01:24:49,621] Trial 24 finished with value: 0.8121011089366286 and parameters: {'n_estimators': 109, 'max_depth': 12, 'min_samples_leaf': 2}. Best is trial 19 with va

Wall time: 2min 59s


In [29]:
# выводим результаты на обучающей выборке
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 100, 'max_depth': 28, 'min_samples_leaf': 2}
f1_score на тестовом наборе: 0.82


In [ ]:
Наилучший результат показала модель Случайный лес с параметрами 
{'max_depth': 20.0, 'min_samples_leaf': 2.0, 'n_estimators': 160.0}.
Данные параметры были подобраны алгоритмом 